In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "snhu1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
        html.A([
            # Add company logo with link to SNHU as well as identifier
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               style={'height': '20%', 'width': '20%'}))]),
    html.Center(html.B(html.H1('CS-340 Dashboard - Brandon Porter'))),
    html.Hr(),
    # Interactive filtering options using radio buttons.
    html.Div(
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label':'Water Rescue', 'value': 'Water'},
                    {'label':'Mountain Rescue', 'value': 'Mountain'},
                    {'label':'Disaster Rescue', 'value': 'Disaster'},
                    {'label':'Reset', 'value': 'default'},
                ],
                value='default',
                labelStyle={'display': 'inline-block'}
            )
        ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
style_table={'height': '500px', 'overflowY': 'auto'},
style_cell={'minWidth': '100px', 'width': '100px', 'maxWidth': '100px'},
filter_action="native",
sort_action="native",
page_size=10
),
html.Br(),
html.Hr(),
html.Div(
    id='map-id',
    className='col s12 m6',
    ),
html.H1("Code Written and Edited by: Brandon Porter"),

#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%', 'height': '100%'},
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%', 'height': '500px'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


## FIX ME Add code to filter interactive data table with MongoDB queries
@app.callback(Output('datatable-id','data'),
              Output('datatable-id', 'selected_rows'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## Filter interactive data table with MongoDB queries
#
    df=pd.DataFrame.from_records(db.read({}))
    selected_rows=[0]
    if filter_type == 'default':
        df = pd.DataFrame.from_records(db.read({}))
    elif filter_type == 'Water':
        df=pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "sex_upon_outcome" : "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte" : 26, "$lte": 156},
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}
        }))
    
    elif filter_type == 'Mountain':
        df=pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte" : 26, "$lte": 156},
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", 
                              "Rottweiler"]}
        }))
        
    elif filter_type == 'Disaster':
        df=pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte" : 20, "$lte": 300},
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound, Rottweiler"]}
        }))
        
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records'), selected_rows

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    # Create pie chart
    df = pd.DataFrame.from_records(viewData)
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    fig = px.pie(df, names='breed', title='Preferred Animals')
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    return dcc.Graph(figure=fig)



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75, -97.48], children=[
                dl.Tooltip(dff.iloc[0, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


connection good
Dash app running on http://127.0.0.1:8050/
